In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
# 设置随机种子
Random.seed!(123)

TaskLocalRNG()

# Parameter

In [ ]:
# 参数设置
S = 10000    # 样本数量
N = 2       # 选项数量
P = 3       # 协变量维度（每个样本的特征维度）

# 真实参数：每个选项 n 有自己的 θ_n ∈ ℝ^P
θ_true = [randn(P)*10 for n in 1:N]  # N × P 的参数列表

In [ ]:
# 生成协变量：每个样本每个选项都有一个特征向量 x_sn ∈ ℝ^P
X = [randn(P) for s in 1:S]  # 长度为 S 的数组，每个元素是 P 维向量

In [ ]:
# 计算效用并生成选择结果 y_s ∈ {1, ..., N}
y = zeros(Int, S)
for s in 1:S
    utility = [dot(θ_true[n], X[s]) for n in 1:N]   # 所有选项共享 x_s
    prob = exp.(utility .- logsumexp(utility))      # softmax
    choice = Categorical(prob) |> rand
    y[s] = choice
end

In [ ]:
# 定义负对数似然函数
function neg_log_likelihood(θ_vec::Vector, X, y; N, P)
    # 将参数向量展开成 N × P 的结构
    Θ = reshape(θ_vec, (P, N))  # 每列是一个 θ_n
    θ_list = [Θ[:, n] for n in 1:N]

    ll = 0.0
    for s in 1:length(y)
        utility = [dot(θ_list[n], X[s]) for n in 1:N]
        log_denom = logsumexp(utility)
        ll += utility[y[s]] - log_denom
    end
    return -ll
end

In [ ]:
# 初始猜测值：N × P 个参数，拉成一个向量
θ_init = zeros(N * P)

# 使用 Optim 进行优化
res = optimize(
    θ -> neg_log_likelihood(θ, X, y, N=N, P=P),
    θ_init,
    BFGS(),
    Optim.Options(iterations=1000, show_trace=false)
)

# 恢复估计出的参数矩阵
θ_hat_matrix = reshape(Optim.minimizer(res), (P, N))
θ_hat_list = [θ_hat_matrix[:, n] for n in 1:N]

# 输出结果
println("真实参数 θ_true:")
for n in 1:N
    println("Option $n: ", θ_true[n])
end

println("\n估计参数 θ_hat:")
for n in 1:N
    println("Option $n: ", θ_hat_list[n])
end

### Predict-then-optimize 

In [2]:
using JuMP
using MosekTools

In [3]:
Random.seed!(123)
# 设置参数
N = 3   # 样本数量（或产品组数）
K = 3   # 每个样本的选择项数量
tau = 10 # 给定常数 τ
N_u = 2

# 预设数据（根据你的实际问题替换这些值）
u = rand(N_u)                         # u 向量 ∈ ℝ^N
A = rand(N,N_u)        # a_n ∈ ℝ^N
B = rand(N,N) .- 1        # b_n ∈ ℝ^N
p_dag = rand(N, K)                  # p_{nk}^† 数据矩阵

3×3 Matrix{Float64}:
 0.0508083  0.835334  0.880073
 0.38644    0.160222  0.784441
 0.846458   0.904766  0.795366

In [10]:
model = Model(Mosek.Optimizer)
# 定义变量
@variable(model, rho_0 >= 0)                          # ρ₀ ≥ 0
@variable(model, rho[1:N] >= 0)                      # ρ_n ≥ 0
@variable(model, v_sigma[1:N_u])                           # ς = ρ₀ * u
@variable(model, v_phi[1:N])                           # φ_n
@variable(model, Y[1:N, 1:K])                      # y_{nk}
@variable(model, Z[1:N, 1:K])                      # z_{nk}
@variable(model, X[1:N, 1:K], Bin)                 # x_{nk} ∈ {0,1}

# 1. 总和约束
@constraint(model, rho_0 + sum(rho) == 1)

# 2. ς = ρ₀ * u
@constraint(model, v_sigma .== rho_0 * u)

# 3. φ_n = ∑_k z_{nk} * p_{nk}^†
@constraint(model, v_phi .== sum(Z .* p_dag, dims=2))

# 4. 辅助变量上下界
for n in 1:N
    for k in 1:K
        # y_{nk} bounds
        @constraint(model, 0 <= Y[n, k])
        @constraint(model, Y[n, k] <= X[n, k])
        @constraint(model, Y[n, k] >= rho[n] - (1 - X[n, k]))
        @constraint(model, Y[n, k] <= rho[n])

        # z_{nk} bounds
        @constraint(model, 0 <= Z[n, k])
        @constraint(model, Z[n, k] <= X[n, k])
        @constraint(model, Z[n, k] >= rho_0 - (1 - X[n, k]))
        @constraint(model, Z[n, k] <= rho_0)
    end
end


# @constraint(model, 1 .== sum(X, dims=2))

# X_given = zeros(N, K)  # 给定的 X 矩阵
# X_given[1, :] = [1, 0, 0]
# X_given[2, :] = [0, 1, 0]   
# X_given[3, :] = [0, 0, 1]
# @constraint(model, X .== X_given)

for n in 1:N
    @constraint(model, [rho[n], rho_0, A[n,:]' * v_sigma + B[n,:]' * v_phi] in MOI.ExponentialCone())
end

@objective(model, Min,sum(Y .* p_dag) - tau * sum(rho))

optimize!(model)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 87              
  Affine conic cons.     : 3 (9 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 36              
  Matrix variables       : 0               
  Integer variables      : 9               

Optimizer started.
Mixed integer optimizer started.
Threads used: 8
Presolve started.
Presolve terminated. Time = 0.00, probing time =  0.00
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  

Objective of best integer solution : Not available.
Best objective bound               : Not available.
Initial feasible solution objective: Undefined
Construct solution objective       : Not employed
User objective cut value           : Not employed
Number of cuts generated    

In [11]:
# 输出状态与目标值
println("求解状态：", termination_status(model))
println("最优目标值：", objective_value(model))

# 输出部分变量（可选）
println("\n估计的变量：")
println("rho_0 = ", value(rho_0))
println("rho = ", value.(rho))
println("X = \n", round.(Int, value.(X)))

求解状态：INFEASIBLE
最优目标值：0.0

估计的变量：
rho_0 = 0.0
rho = [0.0, 0.0, 0.0]
X = 
[0 0 0; 0 0 0; 0 0 0]
